# Projeto 1 - Ciência dos Dados

Nome: Antônio Amaral

Nome: Gabriel Penna

Nome: Caio Garcia

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [522]:
#Definindo Função Magica iPython
%matplotlib inline
#Importanto Pandas
import pandas as pd
#Importando MatPlotLib e Numpy
import matplotlib.pyplot as plt
import numpy as np
#Importando Sistema Operacional
import os
#Importando RE
import re

In [523]:
#Checando Diretório que será trabalhado:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/antonioamaralegydiomartins/Desktop/Antônio/GitHub - Pessoal/INSPER/CDados_Projeto1/Etapa 1 e 2


## Formulas Utilizadas:

In [651]:
def laplace (frase,tabela_rel, tabela_irrel,nao_repetentes,len_rel,len_irrel):
    #Definindo lista de probabilidades
    probR_list=[]
    probIr_list=[]
    #Separando a Frase em elementos
    splt = frase.split()
    #For para reconhecer a quantidade 
    for palavra in splt:
        if palavra not in tabela_rel:
            qtR = 1
        else:
            qtR = tabela_rel[palavra] + 1
        if palavra not in tabela_irrel:
            qtIr = 1
        else:
            qtIr = tabela_irrel[palavra] + 1
        #Definindo divisores
        div1 = len_rel + len(nao_repetentes)
        div2 = len_irrel + len(nao_repetentes)
        #CCalculo de Laplace
        probR_list.append(qtR/div1)
        probIr_list.append(qtIr/div2)
    return (probR_list,probIr_list)

def cleanup(texto):
    temp = texto.lower()
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    temp = re.sub("[^a-z0-9]"," ", temp)
    temp = " ".join(word for word in temp)
    return temp

Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [652]:
#Definindo Arquivo que será trabalhado
filename = 'yakult-final.xlsx'

In [653]:
#Carregando Excel dentro da Variável Trainamento
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Classificação
0,@colchoesueli a médica passou lactobacilos p...,1.0
1,alguém compra yakult pra mim por deus,0.0
2,"amanhã vou fazer aquele drink de morango, ...",0.0
3,tomando um yakult no meio da rua 💪,1.0
4,48 horas e estou sobrevivendo com;\\\\nmei...,1.0


In [654]:
#Carregando Excel dentro da Variável Teste
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Classificação
0,NaN,1
1,"@otavio_amaral18 é uma delícia juro, desce...",1
2,"o cérebro é muito impressionante, ontem eu...",1
3,@fenicolli02 quem q pede gole do yakult???...,0
4,@thamaramaia5 @adrianommartins @taaayrd @t...,0


In [655]:
#Definindo quais são os Tweets com classificação Irrelevante, dentro da Sheet Treinamento
Irrelevantes=train.loc[train['Classificação']==0,'Treinamento']
Irrelevantes

1                alguém compra yakult pra mim por deus  
2          amanhã vou fazer aquele drink de morango, ...
5          @madrugatina sei que vc não bebe, então va...
6          meu lindo :( quero ver ele fazendo batida ...
7          nadei igual um desgraçado, agora comi bata...
                             ...                        
482        talvez eu faça aquela bebida de morango co...
486        meu maior sonho atualmente é um yakult veg...
492        @bcamoezi esses dias comprei couve, alface...
495        ontem eu comi...\\\\npão na chapa\\\\nstak...
497        ontem eu cheguei de madrugada em casa e fi...
Name: Treinamento, Length: 285, dtype: object

In [656]:
#Definindo quais são os Tweets com classificação Relevante, dentro da Sheet Treinamento
Relevantes=train.loc[train['Classificação']==1,'Treinamento'
Relevantes

SyntaxError: invalid syntax (<ipython-input-656-0d41dc546731>, line 3)

___
## Limpeza dos Tweets

In [657]:
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O produto que foi escolhido para ser analizado foi o yakult, a famosa bebida fermentada japonesa, o grupo levou em consideração a emissão de opinião na hora de montar o seu classificador. Sendo assim se houve a emissão de uma uma opinião, sendo benéfica ou não, o Tweet é considerado relevante, porém se houver a emissão de uma opinião que não corresponde ao produto, ou não houve a emissão de uma opinião o Tweet é considerado irrelevante.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

Criando variáveis e realizando limpeza:

In [658]:
#Definindo texto_relevante, como a junção de todos os Tweets relevantes
texto_relevante=' '.join(Relevantes)

In [659]:
#Definindo texto_irrelevante como a junção de todos os Tweets irrelevantes
texto_irrelevante=' '.join(Irrelevantes)

In [660]:
#Passando ambos os textos pela função Cleanup, e convertendo todas as letras para Lowercase
texto_limpo_relevante=cleanup(texto_relevante).lower()
texto_limpo_irrelevante=cleanup(texto_irrelevante).lower()

Convertendo as strings limpas em objetos do tipo pd.series:

In [661]:
palavras_relevantes = texto_limpo_relevante.split()
serie_rel = pd.Series(palavras_relevantes)

In [662]:
palavras_irrelevantes = texto_limpo_irrelevante.split()
serie_irrel = pd.Series(palavras_irrelevantes)

Tornando os objetos transformados acima em series relativas:

In [663]:
serie_irel = serie_rel.value_counts(True)

In [664]:
serie_rrel = serie_irrel.value_counts(True)

In [665]:
#Checando porcentagem das listas
print("Porcentagem de Irrelevantes: {0} e de Relevantes: {1}".format(serie_irel.sum(),serie_rrel.sum()))

Porcentagem de Irrelevantes: 0.9999999999999998 e de Relevantes: 1.0


Montando serie com todas as palavras que aparecem em todos os Tweets:

In [666]:
tabela_rel = serie_rel.value_counts()

In [667]:
tabela_irrel = serie_irrel.value_counts()

In [668]:
tabela_rel_rel = serie_rel.value_counts(True)

In [669]:
tabela_rel_irrel = serie_irrel.value_counts(True)

In [670]:
#Somando Palavras Relevantes e Irrelevantes para criar um dicionário de palavras
todas = texto_limpo_relevante + texto_limpo_irrelevante
#Criando serie_total
total_palavras = todas.split()
serie_total = pd.Series(total_palavras)
#Criando a tabela total de palavras
tabela_total = serie_total.value_counts()

In [671]:
print("Procentagem da Tabela de Irrelevantes: {0} e de Relevantes: {1}".format(tabela_irrel.sum(),tabela_rel.sum()))

Procentagem da Tabela de Irrelevantes: 4361 e de Relevantes: 3271


Adicionando variáveis que representam a soma das tabelas irrel e rel:

In [672]:
RelPal = tabela_rel.sum()


In [673]:
IrrelPal = tabela_irrel.sum()

In [674]:
TotalPal = tabela_total.sum()

Descobrindo a Probabilidade de ser uma palavra Relevante e Irrelevante:

In [675]:
probR=RelPal/TotalPal
print("A probabilidade de uma palavra ser Irrelevante: {0}".format(probI))

A probabilidade de uma palavra ser Irrelevante: 0.5714098532494759


In [676]:
probI=IrrelPal/TotalPal
print("A probabilidade de uma palavra ser Relevante: {0}".format(probR))

A probabilidade de uma palavra ser Relevante: 0.4285901467505241


In [677]:
lista_tabela_total = serie_total.tolist()
nao_repetentes = set(lista_tabela_total)
nao_repetentes = pd.DataFrame(nao_repetentes)

In [686]:
Frase = "quero tomar um  yakult, porque gosto muito e estou viciado"
Frase = cleanup(Frase).lower()
Frase

'quero tomar um  yakult porque gosto muito e estou viciado'

In [687]:
len_rel = len(tabela_rel)
len_irrel = len(tabela_irrel)

In [688]:
probfR = laplace(Frase, tabela_rel, tabela_irrel, nao_repetentes, len_rel, len_irrel)[0]
probfIr = laplace(Frase, tabela_rel, tabela_irrel, nao_repetentes, len_rel, len_irrel)[1]
 
pR = 1
pIr = 1

for e in range(len(probfR)):
    pR = pR * probfR[e]
    pIr = pIr * probfIr[e]

In [689]:
if pR > pIr:
    print("Relevante")
else:
    print("Irrelevante")

Relevante


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**